In [1]:
import math
import os
import keras as k
import numpy as np
import pandas as pd

/home/sai/code/personal/py-snippets/.env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model = k.applications.VGG16(weights = 'imagenet',include_top=False)
x = model.get_layer('block5_conv3').output
x = k.layers.GlobalAveragePooling2D()(x)
x = k.layers.Dense(256, activation='relu')(x)
x = k.layers.Dropout(0.5)(x)
x = k.layers.Dense(1, activation='sigmoid')(x)
model_final = k.models.Model(inputs=model.input, outputs=x)

In [3]:
for layer in model.layers:
    layer.trainable = False

In [4]:
model_final.compile(optimizer = k.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True),
                    loss      = 'binary_crossentropy',
                    metrics   = ['accuracy'])

In [5]:
from keras.preprocessing.image import ImageDataGenerator

In [6]:
datagen = ImageDataGenerator( 
    featurewise_center            = True,
    rescale                       = 1.,
    rotation_range                = 10,
    width_shift_range             = .1,
    height_shift_range            = .1,
    shear_range                   = 0.2,
    zoom_range                    = 0.2,
    horizontal_flip               = False,
    vertical_flip                 = False,
    fill_mode                     = "reflect")

# normalization neccessary for correct image input to VGG16
datagen.mean=np.array([103.939, 116.779, 123.68], dtype=np.float32).reshape(1,1,3)

# no data augmentation for validation and test set
validgen = ImageDataGenerator(rescale=1., featurewise_center=True)
validgen.mean=np.array([103.939, 116.779, 123.68], dtype=np.float32).reshape(1,1,3)

In [7]:
img_width  = 512
img_height = 512

train_data_dir      = "data/train2"
validation_data_dir = "data/valid2"
test_data_dir       = "data/test2"

batch_size_train = 8
batch_size_val   = 8

train_gen = datagen.flow_from_directory(
        directory   = train_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_train,
        class_mode  = "binary",
        shuffle     = True)

val_gen = validgen.flow_from_directory(
        directory   = validation_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_val,
        class_mode  = "binary",
        shuffle     = False)

test_gen = validgen.flow_from_directory(
        directory   = test_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_val,
        class_mode  = "binary",
        shuffle     = False)

train_samples      = len(train_gen.filenames)
validation_samples = len(val_gen.filenames)
test_samples       = len(test_gen.filenames)

Found 1312 images belonging to 2 classes.
Found 524 images belonging to 2 classes.
Found 788 images belonging to 2 classes.


In [8]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=15,
                           verbose=1,
                           min_delta=1e-4,
                           mode='min'),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=3,
                               verbose=1,
                               epsilon=1e-4,
                               mode='min'),
             ModelCheckpoint(monitor='val_loss',
                             filepath='data/weights/best_weights_m2.hdf5',
                             save_best_only=True,
                             save_weights_only=True,
                             mode='min')]

In [9]:
model_final.fit_generator(generator        = train_gen,
                          epochs           = 50, 
                          steps_per_epoch  = math.ceil(train_samples / batch_size_train), 
                          validation_data  = val_gen, 
                          validation_steps = math.ceil(validation_samples / batch_size_val), 
                          verbose          = 2,
                          callbacks        = callbacks,
                          class_weight     = {0: 96/1312, 1: 1216/1312})

Epoch 1/50
 - 129s - loss: 0.1071 - acc: 0.4840 - val_loss: 0.5996 - val_acc: 0.6985
Epoch 2/50
 - 124s - loss: 0.0929 - acc: 0.6021 - val_loss: 0.6349 - val_acc: 0.6393
Epoch 3/50
 - 124s - loss: 0.0840 - acc: 0.6562 - val_loss: 0.6256 - val_acc: 0.6527
Epoch 4/50
 - 129s - loss: 0.0835 - acc: 0.6524 - val_loss: 0.6780 - val_acc: 0.6164
Epoch 5/50
 - 123s - loss: 0.0826 - acc: 0.6761 - val_loss: 0.6031 - val_acc: 0.6756

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 6/50
 - 127s - loss: 0.0766 - acc: 0.6845 - val_loss: 0.6169 - val_acc: 0.6565
Epoch 7/50
 - 127s - loss: 0.0748 - acc: 0.6845 - val_loss: 0.5963 - val_acc: 0.6775
Epoch 8/50
 - 121s - loss: 0.0810 - acc: 0.6822 - val_loss: 0.6041 - val_acc: 0.6679
Epoch 9/50
 - 124s - loss: 0.0756 - acc: 0.6806 - val_loss: 0.5831 - val_acc: 0.6908
Epoch 10/50
 - 124s - loss: 0.0735 - acc: 0.7134 - val_loss: 0.5833 - val_acc: 0.6870
Epoch 11/50
 - 124s - loss: 0.0818 - acc: 0.6913 - val_loss: 0.5994

KeyboardInterrupt: 

In [14]:
model_final.load_weights('data/weights/best_weights_m2.hdf5')

In [15]:
for layer in model_final.layers:
    layer.trainable = True

In [16]:
model_final.compile(optimizer = k.optimizers.SGD(lr=0.000001, momentum=0.9, nesterov=True),
                    loss      = 'binary_crossentropy',
                    metrics   = ['accuracy'])

In [18]:
model_final.fit_generator(generator        = train_gen,
                          epochs           = 50, 
                          steps_per_epoch  = math.ceil(train_samples / batch_size_train), 
                          validation_data  = val_gen, 
                          validation_steps = math.ceil(validation_samples / batch_size_val), 
                          verbose          = 2,
                          callbacks        = callbacks,
                          class_weight     = {0: 96/1312, 1: 1216/1312})

Epoch 1/50


ResourceExhaustedError: OOM when allocating tensor with shape[8,64,512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_2/SGD/gradients/block1_conv2/convolution_grad/Conv2DBackpropInput = Conv2DBackpropInput[T=DT_FLOAT, _class=["loc:@train...propFilter"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_2/SGD/gradients/block1_conv2/convolution_grad/ShapeN, block1_conv2/kernel/read, training_2/SGD/gradients/block1_conv2/Relu_grad/ReluGrad)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training_2/SGD/gradients/block1_conv2/convolution_grad/Conv2DBackpropInput', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/p

In [ ]:
model_final.load_weights('data/weights/best_weights.hdf5')

In [ ]:
test_gen = validgen.flow_from_directory(
        directory   = test_data_dir,
        target_size = (img_height, img_width),
        batch_size  = 32,
        class_mode  = "binary",
        shuffle     = False)
test_samples = len(test_gen.filenames)
preds = model_final.predict_generator(test_gen, math.ceil(test_samples / 32))

In [ ]:
preds = [x[0] for x in preds]

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(test_gen.classes, preds)

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(test_gen.classes, preds)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(7,7))
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve')
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()